## DeZeroのMNISTに畳み込み層とプーリング層とDropoutを追加して実験
「[ゼロから作るDeep Learning ❸](https://www.oreilly.co.jp/books/9784873119069/)ステップ51 MNISTの学習」の[`deep-learning-from-scratch-3/examples/mnist.py`](https://github.com/oreilly-japan/deep-learning-from-scratch-3/blob/master/examples/mnist.py)は、次のようなネットワーク構成になっています。

 - `(1, 28, 28)入力→(784,)配列→Linear(1000)→ReLU→Linear(1000)→ReLU→Linear(10)出力`
 - 最適化手法：Adam

学習経過は次のようになります。

```bash
$ python -m examples.mnist
epoch: 1
train loss: 0.19297177767846732, accuracy: 0.9425166666666667
test loss: 0.0870012722350657, accuracy: 0.972
epoch: 2
train loss: 0.07911744002679673, accuracy: 0.9756333333333334
test loss: 0.08153347594663501, accuracy: 0.9735
epoch: 3
train loss: 0.05772087200079113, accuracy: 0.9810833333333333
test loss: 0.06439740799571154, accuracy: 0.9804
epoch: 4
train loss: 0.04381801278757242, accuracy: 0.9858
test loss: 0.07971655959896452, accuracy: 0.9763
epoch: 5
train loss: 0.03815114084631205, accuracy: 0.9882666666666666
test loss: 0.07151200462241832, accuracy: 0.9778
```

これに次のような、標準的な畳み込み層、プーリング層、ドロップアウトを追加したモデルを作成します。

 - `(1, 28, 28)入力→Conv→ReLU→Pool(2)→Linear(1000)→ReLU→Dropout→Linear(1000)→ReLU→Dropout→Linear(10)出力`
   - `Conv`は、カーネルサイズ=3、ストライドサイズ=1、パディングサイズ=1
   - 入力データサイズ28 + 2 * パディングサイズ1 - カーネルサイズ3 // ストライドサイズ1 + 1 = 出力サイズ28
   - 2x2プーリングなので全結合層`Linear(1000)`に入る時点で縦横半分の14x14となる。

実行は、`pip install dezero`（Jupyterなら`!pip install dezero`）でDeZeroをインストールして次を実行するか、
[`qitqito/dezero_study/mnist_plus.py`](https://github.com/qitqito/dezero_study/blob/master/mnist_plus.py)をダウンロードして`python -m mnist_plus`などして下さい。

In [1]:
import dezero
import dezero.functions as F
from dezero import DataLoader
from dezero.models import Model
import dezero.layers as L


class MNISTPlus(Model):
    def __init__(self, hidden_size=100):
        super().__init__()
        self.conv1 = L.Conv2d(1, kernel_size=3, stride=1, pad=1)
        #self.conv2 = L.Conv2d(1, kernel_size=3, stride=1, pad=1)
        self.fc3 = L.Linear(hidden_size)
        self.fc4 = L.Linear(hidden_size)
        self.fc5 = L.Linear(10)

    def forward(self, x):
        x = F.relu(self.conv1(x)) # 28x28
        x = F.pooling(x, 2, 2) # 14x14
        #x = F.relu(self.conv2(x))
        #x = F.pooling(x, 2, 2)
        x = F.reshape(x, (x.shape[0], -1)) # 14x14を196に
        x = F.dropout(F.relu(self.fc3(x)))
        x = F.dropout(F.relu(self.fc4(x)))
        x = self.fc5(x)
        return x

max_epoch = 20
batch_size = 100

train_set = dezero.datasets.MNIST(train=True, transform=None) # 28x28のまま
test_set = dezero.datasets.MNIST(train=False, transform=None) # 28x28のまま
train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MNISTPlus(1000)
optimizer = dezero.optimizers.Adam().setup(model)
optimizer.add_hook(dezero.optimizers.WeightDecay(1e-4))  # Weight decay

if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    test_loader.to_gpu()
    model.to_gpu()

for epoch in range(max_epoch):
    sum_loss, sum_acc = 0, 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        acc = F.accuracy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()

        sum_loss += float(loss.data) * len(t)
        sum_acc += float(acc.data) * len(t)

    print('epoch: {}'.format(epoch+1))
    print('train loss: {}, accuracy: {}'.format(
        sum_loss / len(train_set), sum_acc / len(train_set)))

    sum_loss, sum_acc = 0, 0
    with dezero.no_grad():
        for x, t in test_loader:
            y = model(x)
            loss = F.softmax_cross_entropy(y, t)
            acc = F.accuracy(y, t)
            sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)

    print('test loss: {}, accuracy: {}'.format(
        sum_loss / len(test_set), sum_acc / len(test_set)))

epoch: 1
train loss: 3.2967599945267043, accuracy: 0.7726333333333333
test loss: 0.6105298531055451, accuracy: 0.841
epoch: 2
train loss: 0.4918030514443914, accuracy: 0.8647833333333333
test loss: 0.39708854403346777, accuracy: 0.8898
epoch: 3
train loss: 0.365787100456655, accuracy: 0.8968166666666667
test loss: 0.3391792545514181, accuracy: 0.9072
epoch: 4
train loss: 0.31472394374509655, accuracy: 0.9104833333333333
test loss: 0.33774808773770926, accuracy: 0.9031
epoch: 5
train loss: 0.2928329927722613, accuracy: 0.9176166666666666
test loss: 0.30439508248120545, accuracy: 0.9157
epoch: 6
train loss: 0.2737082013487816, accuracy: 0.9220666666666667
test loss: 0.2845079012773931, accuracy: 0.9228
epoch: 7
train loss: 0.24680014058947564, accuracy: 0.92935
test loss: 0.28378744328394534, accuracy: 0.9223
epoch: 8
train loss: 0.23471303268025318, accuracy: 0.9321666666666667
test loss: 0.27566407042555513, accuracy: 0.9266
epoch: 9
train loss: 0.21447662274663648, accuracy: 0.9387333

### 考察
全結合層に畳み込み層・プーリング層・Dropoutを追加したら学習性能が悪化しました。

MNISTの画像が小さ過ぎて畳み込みやプーリングが情報を減らしてしまうのかも知れません。Dropoutも有無で実験したところ学習を遅くしてるように思われます。全結合層は1000より小さくしても良いかも知れません。。。

自分には、知識不足でろくな考察ができないと分かったので、当分は深層学習などを色々学んでいき、ブログ一記事分たまったらその都度書いていこうと思います。